# Create Spectrograms

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
from scipy import signal
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import librosa.display

import numpy as np
import pandas as pd
import librosa
emotions = ['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD']
d = dict([(y,x) for x,y in enumerate(sorted(set(emotions)))])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = os.fsencode("/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/CREMA-D")

# ANG, DIS, FEA, HAP, NEU, SAD
# 0,   1,   2,   3,   4,   5

count = 0

for file in os.listdir(directory):
  filename = os.fsdecode(file)
  a = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/CREMA-D/"
  b = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/spec/"
  y, sr = librosa.load(a + filename)
  y = y[:100000] # shorten audio a bit for speed

  window_size = 1024
  window = np.hanning(window_size)
  stft  = librosa.core.spectrum.stft(y, n_fft=window_size, hop_length=512, window=window)
  out = 2 * np.abs(stft) / np.sum(window)

  # For plotting headlessly
  from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

  fig = plt.Figure()
  canvas = FigureCanvas(fig)
  ax = fig.add_subplot(111)
  p = librosa.display.specshow(librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis='log', x_axis='time')
  emotion = filename[9:12]
  fig.savefig(b + emotion + str(count) + ".jpg")
  count += 1

# Create Dataset

In [ ]:
import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO

def crop(image):
  return image.crop((80, 58, 577, 428))

transform = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Lambda(crop),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


Just a sanity check

In [ ]:
import matplotlib.image as mpimg

direct = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/spec/"
cntt = 0
for file in os.listdir(direct):
  img = mpimg.imread(direct + file)
  plt.imshow(img)
  plt.show()
  with open(direct + file, 'rb') as f:
    img = f.read()
    image = Image.open(BytesIO(img))
    image = transform(image)
  plt.imshow(image)
  plt.axis('off')
  plt.show()
  cntt += 1
  #print(cntt)
  if cntt == 5:
    break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
root_path = "/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/"
folder = "spec/"

directory = os.fsencode(root_path + folder)
count = 0

features = []
labels = []
cnt = 0

for file in os.listdir(directory):
  filename = os.fsdecode(file)
  with open(root_path + folder + filename, 'rb') as f:
    img = f.read()
    image = Image.open(BytesIO(img))
    image = transform(image)
    features.append(image)
  emote = filename[0:3]
  labels.append(torch.tensor([d[emote]]))

In [ ]:
features = torch.stack(features)
labels = torch.stack(labels)

In [ ]:
dset = torch.utils.data.TensorDataset(features, labels)

In [ ]:
torch.save(dset, "dset.pt")

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

train_loader = DataLoader(dset, batch_size = 61, shuffle = True)

In [ ]:
!cp pipe.pkl /content/drive/MyDrive/Machine Learning/dset.pt

cp: target 'Learning/name' is not a directory
